###### The  data  has  the  following  two  tables:
###### 1]  A  user  table  ( "takehome_users" )  with  data  on  12,000  users  who  signed  up  for  the product  in  the  last  two  years.   
###### 2]  A  usage  summary  table  ( "takehome_user_engagement" )  that  has  a  row  for  each  day that  a  user  logged  into  the  product.
###### Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user adoption .

In [555]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime, date, timedelta

In [556]:
#load takehome_user_engagement data file
da = pd.read_csv('takehome_user_engagement.csv')
da.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [628]:
#import takehome_users.csv datafile
db = pd.read_csv('takehome_users.csv', encoding='latin-1', index_col=0)
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 9 columns):
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(3), object(4)
memory usage: 937.5+ KB


In [629]:
db.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [630]:
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 4 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
adopted       207917 non-null bool
dtypes: bool(1), datetime64[ns](1), int64(2)
memory usage: 5.0 MB


In [631]:
#convert datetime to only date
da['time_stamp'] = pd.to_datetime(da['time_stamp'])
# da.sort_values('time_stamp')
dz = da.set_index('time_stamp')
dz

,user_id,visited,adopted
time_stamp,,,
2014-04-22 03:53:30,1,1,False
2013-11-15 03:45:04,2,1,False
2013-11-29 03:45:04,2,1,False
2013-12-09 03:45:04,2,1,False
2013-12-25 03:45:04,2,1,False
...,...,...,...
2013-09-06 06:14:15,11996,1,False
2013-01-15 18:28:37,11997,1,False
2014-04-27 12:45:16,11998,1,False


In [632]:
d = dz.groupby('user_id').resample('1W').sum()
d

user_id  visited  adopted
user_id time_stamp                           
1       2014-04-27        1        1      0.0
2       2013-11-17        2        1      0.0
        2013-11-24        0        0      0.0
        2013-12-01        2        1      0.0
        2013-12-08        0        0      0.0
...                     ...      ...      ...
11996   2013-09-08    11996        1      0.0
11997   2013-01-20    11997        1      0.0
11998   2014-04-27    11998        1      0.0
11999   2012-06-03    11999        1      0.0
12000   2014-01-26    12000        1      0.0

[82282 rows x 3 columns]

In [633]:
d_adopted = d[d['visited']>2]
del d_adopted['user_id']
d_adopted

visited  adopted
user_id time_stamp                  
2       2014-02-09        3      0.0
10      2013-03-03        3      0.0
        2013-04-14        3      0.0
        2013-04-28        3      0.0
        2013-05-05        4      0.0
...                     ...      ...
11988   2014-04-06        3      0.0
        2014-05-04        4      0.0
        2014-05-18        3      0.0
        2014-05-25        5      0.0
        2014-06-01        3      0.0

[33859 rows x 2 columns]

In [634]:
# create user id list for adopted users
adopted_index = list(d_adopted.index)
adopted_users_list = []
for i in range(len(adopted_index)):
    adopted_users_list.append(adopted_index[i][0])
len(adopted_users_list)

33859

In [635]:
# remove duplicate user ids
adopted_users = set(adopted_users_list)
len(adopted_users)

1445

In [636]:
# add an adopted column to the dataframe with a default value of False
db['adopted'] = False

# use the adopted_users set to label the adopted users in the dataframe
for user in adopted_users:
    db.loc[user, 'adopted'] = True

db.head(10)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted
object_id,,,,,,,,,,
1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,False
2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,True
3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,False
4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,False
5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,False
6,12/17/2013 3:37,Cunha Eduardo,EduardoPereiraCunha@yahoo.com,GUEST_INVITE,1.387424e+09,0,0,197,11241.0,False
7,12/16/2012 13:24,Sewell Tyler,TylerSewell@jourrapide.com,SIGNUP,1.356010e+09,0,1,37,NaN,False
8,7/31/2013 5:34,Hamilton Danielle,DanielleHamilton@yahoo.com,PERSONAL_PROJECTS,NaN,1,1,74,NaN,False
9,11/5/2013 4:04,Amsel Paul,PaulAmsel@hotmail.com,PERSONAL_PROJECTS,NaN,0,0,302,NaN,False


In [651]:
db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 10 columns):
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
adopted                       12000 non-null bool
invited_by_user               12000 non-null bool
dtypes: bool(2), float64(1), int64(3), object(4)
memory usage: 1.2+ MB


In [652]:
db['invited_by_user_id'].value_counts(ascending=False, dropna=False)

KeyError: 'invited_by_user_id'

In [ ]:
db['invited_by_user'] = db['invited_by_user_id'].isnull() == False
del db['invited_by_user_id']
db.head(10)

In [ ]:
# split the individual email address from email domain
db['email_split'] = db['email'].str.split('@')
del df_users['email']

# extract the email domains into separate column
for user in range(len(db)):
    db.loc[user+1, 'email_domain'] = db.loc[user+1, 'email_split'][1]
del db['email_split']

# evaluate the most common email domains
db['email_domain'].value_counts()[:10]

In [ ]:
db['email'] = 'other'

# populate the final email column with the common email domains
for user in range(len(db)):
    if db.loc[user+1, 'email_domain'] == 'gmail.com':
        db.loc[user+1, 'email'] = 'gmail.com'
    if db.loc[user+1, 'email_domain'] == 'yahoo.com':
        db.loc[user+1, 'email'] = 'yahoo.com'
    if db.loc[user+1, 'email_domain'] == 'jourrapide.com':
        db.loc[user+1, 'email'] = 'jourrapide.com'
    if db.loc[user+1, 'email_domain'] == 'cuvox.de':
        db.loc[user+1, 'email'] = 'cuvox.de'
    if db.loc[user+1, 'email_domain'] == 'gustr.com':
        db.loc[user+1, 'email'] = 'gustr.com'
    if db.loc[user+1, 'email_domain'] == 'hotmail.com':
        db.loc[user+1, 'email'] = 'hotmail.com'

del db['email_domain']

db['email'].value_counts(ascending=False)

In [653]:
db.head()

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted,invited_by_user
object_id,,,,,,,,,,
1,4/22/2014 3:53,Clausen August,yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,False,True
2,11/15/2013 3:45,Poole Matthew,gustr.com,ORG_INVITE,1.396238e+09,0,0,1,True,True
3,3/19/2013 23:14,Bottrill Mitchell,gustr.com,ORG_INVITE,1.363735e+09,0,0,94,False,True
4,5/21/2013 8:09,Clausen Nicklas,yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,False,True
5,1/17/2013 10:14,Raw Grace,yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,False,True


In [654]:
#Data Preparation 
df_ml = pd.DataFrame({'adopted_user': db['adopted'].values}, index=db.index)
df_ml

,adopted_user
object_id,
1,False
2,True
3,False
4,False
5,False
...,...
11996,False
11997,False
11998,False


In [655]:
#Binary Features
df_ml['mailing_list'] = db['opted_in_to_mailing_list']
df_ml['marketing_drip'] = db['enabled_for_marketing_drip']
df_ml['invited_by_user'] = db['invited_by_user']
df_ml

,adopted_user,mailing_list,marketing_drip,invited_by_user
object_id,,,,
1,False,1,0,True
2,True,0,0,True
3,False,0,0,True
4,False,0,0,True
5,False,0,0,True
...,...,...,...,...
11996,False,0,0,True
11997,False,0,0,False
11998,False,1,1,True


In [656]:
#Categorical Variables
# convert org_id column from int to string
db['org_id'] = db['org_id'].astype('str')

# create the dummy variables
df_dummies = pd.get_dummies(db[['creation_source', 'org_id', 'email']])

# add dummy variables to df_model dataframe
df_ml = pd.concat([df_ml, df_dummies], axis=1)
df_ml.head(3)

,adopted_user,mailing_list,marketing_drip,invited_by_user,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,org_id_0,...,org_id_97,org_id_98,org_id_99,email_cuvox.de,email_gmail.com,email_gustr.com,email_hotmail.com,email_jourrapide.com,email_other,email_yahoo.com
object_id,,,,,,,,,,,,,,,,,,,,,
1,False,1,0,True,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,True,0,0,True,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,False,0,0,True,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [657]:
#Predictive Modeling
X = df_ml.iloc[:, 1:]
y = df_ml.loc[:, 'adopted_user']

# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=6, stratify=y)

In [658]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced_subsample', random_state=6)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_test)
print('Accuracy on training set = {}'.format(rfc.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc.score(X_test, y_test)))


Accuracy on training set = 0.9427777777777778
Accuracy on test set = 0.837


In [659]:
# generate classification report of model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc))

              precision    recall  f1-score   support

       False       0.88      0.94      0.91      2639
        True       0.18      0.10      0.13       361

    accuracy                           0.84      3000
   macro avg       0.53      0.52      0.52      3000
weighted avg       0.80      0.84      0.82      3000



In [660]:
# generate confusion matrix for model performance
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc))

[[2476  163]
 [ 326   35]]


In [661]:
# create dataframe of feature importances from model
df_features = pd.DataFrame({'rfc': rfc.feature_importances_}, index=df_ml.columns[1:])

# sort by highest values
df_features.sort_values('rfc', ascending=False)[:15]

,rfc
mailing_list,0.036834
marketing_drip,0.025103
email_gmail.com,0.016393
email_yahoo.com,0.013526
email_gustr.com,0.013521
email_jourrapide.com,0.012945
email_other,0.012826
creation_source_PERSONAL_PROJECTS,0.011557
email_cuvox.de,0.010256
email_hotmail.com,0.009893


In [662]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rfc = RandomForestClassifier(random_state=6)
param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [5, 15, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 3, 5]}
grid = GridSearchCV(rfc, param_grid, cv=3, scoring='accuracy')
grid.fit(X_train, y_train)
y_pred_rfc = grid.predict(X_test)
print(grid.score(X_test, y_test))

0.8796666666666667


In [663]:

grid.best_score_

0.8795555555555555

In [664]:
grid.best_estimator_

RandomForestClassifier(max_depth=5, random_state=6)

In [665]:
#Tuned Model Performance
# train the model and make predictions on the training and test sets
from sklearn.ensemble import RandomForestClassifier
rfc_tuned = RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, min_samples_leaf=1, criterion='gini', class_weight='balanced_subsample', random_state=6)
rfc_tuned.fit(X_train, y_train)
y_pred_rfc_tuned = rfc_tuned.predict(X_test)
print('Accuracy on training set = {}'.format(rfc_tuned.score(X_train, y_train)))
print('Accuracy on test set = {}'.format(rfc_tuned.score(X_test, y_test)))

Accuracy on training set = 0.6206666666666667
Accuracy on test set = 0.6


In [666]:

# generate classification report of model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_rfc_tuned))

              precision    recall  f1-score   support

       False       0.90      0.61      0.73      2639
        True       0.15      0.51      0.23       361

    accuracy                           0.60      3000
   macro avg       0.53      0.56      0.48      3000
weighted avg       0.81      0.60      0.67      3000



In [667]:
# generate confusion matrix for model performance
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_rfc_tuned))

[[1616 1023]
 [ 177  184]]


In [668]:
# add the tuned classifier feature importance values to the df_features dataframe
df_features['rfc_tuned'] = rfc_tuned.feature_importances_

# add column calculating the difference between the feature importance values
df_features['difference'] = df_features['rfc_tuned'] - df_features['rfc']

# sort by highest difference
df_features.sort_values('difference', ascending=False)[:15]

,rfc,rfc_tuned,difference
creation_source_PERSONAL_PROJECTS,0.011557,0.080421,0.068864
org_id_0,0.008809,0.065023,0.056215
creation_source_GUEST_INVITE,0.008469,0.063778,0.055309
email_yahoo.com,0.013526,0.044497,0.030971
email_hotmail.com,0.009893,0.037771,0.027877
creation_source_SIGNUP_GOOGLE_AUTH,0.005455,0.022810,0.017355
email_gmail.com,0.016393,0.033124,0.016731
org_id_1,0.007304,0.022089,0.014784
org_id_3,0.006190,0.017236,0.011046
creation_source_ORG_INVITE,0.009336,0.019857,0.010521
